In [3]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('lang=ko_KR') # 사용언어 한국어
options.add_argument('disable-gpu') # 하드웨어 가속 안함
# options.add_argument('headless') # 창 숨기기


In [4]:
import user_agent
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))
# print(generate_user_agent(os=('mac', 'linux'), device_type='desktop'))

navigator = generate_navigator()
# print(navigator)
# print(navigator['platform'])

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

Mozilla/5.0 (X11; Ubuntu; Linux i686 on x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; rv:47.0) Gecko/20100101 Firefox/47.0
Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.110 Safari/537.36


In [7]:
def scroll_first():
    #### 처음 제목, 링크, 날짜 리스트에 담기
    #셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
    prev_height = driver.execute_script("return document. body.scrollHeight")

    while True:
        #첫번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #시간대기
        time.sleep(2)
        #현재높이 저장
        current_height = driver.execute_script("return document. body.scrollHeight")
        #현재높이와 끝의 높이가 같으면 탈출
        if(current_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

    # 검색 결과 블로그글 url과 제목 가져오기
    class_articles = ".api_txt_lines.total_tit"
    url_link = driver.find_elements(By.CSS_SELECTOR, class_articles)
    # 글 작성 일자 가져오기
    class_datetime = ".sub_time.sub_txt"
    date_time = driver.find_elements(By.CSS_SELECTOR, class_datetime)
    
    global url_list
    global title_list
    global date_list

    url_list = []
    title_list = []
    date_list = []

    for article in url_link:
        url = article.get_attribute('href')
        url_list.append(url)
    for article in url_link:
        title = article.text
        title_list.append(title)
    for date in date_time:
        datetime = date.text
        date_list.append(datetime)

In [8]:
def scroll_next():
    #셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
    prev_height = driver.execute_script("return document. body.scrollHeight")

    while True:
        #첫번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #시간대기
        time.sleep(2)
        #현재높이 저장
        current_height = driver.execute_script("return document. body.scrollHeight")
        #현재높이와 끝의 높이가 끝이면 탈출
        if(current_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

    # 검색 결과 블로그글 url과 제목 가져오기
    class_articles = ".api_txt_lines.total_tit"
    url_link = driver.find_elements(By.CSS_SELECTOR, class_articles)
    # 글 작성 일자 가져오기
    class_datetime = ".sub_time.sub_txt"
    date_time = driver.find_elements(By.CSS_SELECTOR, class_datetime)

    for article in url_link:
        url = article.get_attribute('href')
        url_list.append(url)
    for article in url_link:
        title = article.text
        title_list.append(title)
    for date in date_time:
        datetime = date.text
        date_list.append(datetime)

In [9]:
# year=연도(연도에 지정된 숫자! 2018년은 16), month=월(1~12), days=해당 월의 일수
def basic_crawling(year, month, days):
    day_list = list(range(days,0,-1))
    
    driver.find_element(By.LINK_TEXT, "VIEW").click()
    time.sleep(0.5)
    driver.find_element(By.LINK_TEXT, "블로그").click()
    time.sleep(0.5)
    driver.find_element(By.LINK_TEXT, "옵션").click()
    time.sleep(0.5)
    driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.lineup > div > div > a:nth-child(2)").click()
    time.sleep(0.5)
    
    # 옵션 > 기간 >  직접입력 클릭
    driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger").click()
    time.sleep(0.5)

    # 기간 직접입력 시작과 끝의 앞부분 동일 CSS 코드 변수로 담기 (코드 단순화용)
    GS_head_css = "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > "
    # 기간 직접입력 년월일 앞 CSS 코드 변수로 담기 (코드 단순화용)
    GSYMD_head_css = "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > "
    
    for day in day_list:
        if day == days:
            print('{}일 작성글 검색 후 수집중 . . .'.format(day))
        #  ★☆★☆ 맨 처음 기간설정 할 때 ☆★☆★
            #### 기간 설정시작 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(1) > a").click()
            time.sleep(0.5)
            # 연도는 year
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(1) > div > div > div > ul > li:nth-child({})".format(year)).click()
            time.sleep(0.5)
            # 월은 month
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(2) > div > div > div > ul > li:nth-child({})".format(month)).click()
            time.sleep(0.5)
            # 일자는 days. 마지막 일자 부터 검색해서 저장할 것이기 때문!
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(days)).click()
            time.sleep(0.5)

            #### 기간 설정끝 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(3) > a").click()
            time.sleep(0.5)
            # 연도는 year
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(1) > div > div > div > ul > li:nth-child({})".format(year)).click()
            time.sleep(0.5)
            # 월은 month
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(2) > div > div > div > ul > li:nth-child({})".format(month)).click()
            time.sleep(0.5)
            # 일자는 days. 마지막 일자 부터 검색해서 저장할 것이기 때문!
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(days)).click()
            time.sleep(0.5)

            #### 기간 설정란 적용 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.btn_area > button").click()
            time.sleep(0.5)
            
            # 첫 번째 스크롤&전역변수 선언&제목/링크/날짜 저장
            scroll_first()

        
        else:
        # 두 번째부터는 스크롤 맨 위로 올려서 일자만 바꾸기!
            print('{}일 작성글 검색 후 수집중 . . .'.format(day))
            # 스크롤 맨 위로 올리기
            driver.execute_script('window.scrollTo(0,0)')
            time.sleep(1)
            # 옵션 > 기간 >  직접입력 클릭
            driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger").click()
            driver.implicitly_wait(10)
            # 기간 설정시작 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(1) > a").click()
            driver.implicitly_wait(10)
            # # 일자는 day_list에서 for 문 돌면서 i번째 숫자 가져오기
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(day)).click()
            driver.implicitly_wait(10)
            # 기간 설정끝 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.set_calendar > span:nth-child(3) > a").click()
            driver.implicitly_wait(10)
            # # 일자는 day_list에서 for 문 돌면서 i번째 숫자 가져오기
            driver.find_element(By.CSS_SELECTOR, GSYMD_head_css + "div:nth-child(3) > div > div > div > ul > li:nth-child({})".format(day)).click()
            # 기간 설정란 적용 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, GS_head_css + "div.btn_area > button").click()
            driver.implicitly_wait(10)
            
            # 두 번째 스크롤&제목/링크/날짜 저장
            scroll_next()
            
    global crawl_df
    crawl_df = pd.DataFrame({'url': url_list, 'title':title_list, 'date':date_list})

In [10]:
Yr = 18 # 2020년도는 18
for i in range(6,0,-1): # 7부터 1까지 차례대로 반복
    chrome_options = Options()
    chrome_options.add_argument(random_user)
    driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)

    driver.maximize_window()
    driver.get("https://www.naver.com/")
    driver.implicitly_wait(10)

    query_text = '"나무" +"문화"'

    element = driver.find_element(By.ID, "query");

    element.send_keys(query_text)
    element.submit()
    driver.implicitly_wait(10)

    if i in [1,3,5,7,8,10,12]:
        basic_crawling(Yr,i,31)
    elif i in [4,6,9,11]:
        basic_crawling(Yr,i,30)
    elif i == 2 and (Yr+2)%4 == 0: # 윤년을 고려하여 2월 29일인 케이스 추가
         basic_crawling(Yr,i,29)
    elif i == 2 and (Yr+2)%4 != 0: # 윤년이 아닌 2월일 경우
        basic_crawling(Yr,i,28)
    crawl_df.drop_duplicates(inplace=True) #중복제거
    crawl_df = crawl_df.reset_index(drop=True) #인덱스 초기화 

    link_list = crawl_df['url'].to_list()
    find_naver = [i for i in range(len(link_list)) if 'naver' in link_list[i]]

    NB_DF = crawl_df.iloc[find_naver]

    NB_DF.to_csv("./나무문화/2020/나무문화_링크_2020_%s월.csv"%i, index=False, encoding='UTF-8')
    NB_DF.to_excel("./나무문화/2020/나무문화_링크_2020_%s월.xlsx"%i, index=False, encoding='UTF-8')
    driver.quit()
    print(i)

C:\Users\user\AppData\Local\Temp\ipykernel_13528\1250680245.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)


30일 작성글 검색 후 수집중 . . .
29일 작성글 검색 후 수집중 . . .
28일 작성글 검색 후 수집중 . . .
27일 작성글 검색 후 수집중 . . .
26일 작성글 검색 후 수집중 . . .
25일 작성글 검색 후 수집중 . . .
24일 작성글 검색 후 수집중 . . .
23일 작성글 검색 후 수집중 . . .
22일 작성글 검색 후 수집중 . . .
21일 작성글 검색 후 수집중 . . .
20일 작성글 검색 후 수집중 . . .
19일 작성글 검색 후 수집중 . . .
18일 작성글 검색 후 수집중 . . .
17일 작성글 검색 후 수집중 . . .
16일 작성글 검색 후 수집중 . . .
15일 작성글 검색 후 수집중 . . .
14일 작성글 검색 후 수집중 . . .
13일 작성글 검색 후 수집중 . . .
12일 작성글 검색 후 수집중 . . .
11일 작성글 검색 후 수집중 . . .
10일 작성글 검색 후 수집중 . . .
9일 작성글 검색 후 수집중 . . .
8일 작성글 검색 후 수집중 . . .
7일 작성글 검색 후 수집중 . . .
6일 작성글 검색 후 수집중 . . .
5일 작성글 검색 후 수집중 . . .
4일 작성글 검색 후 수집중 . . .
3일 작성글 검색 후 수집중 . . .
2일 작성글 검색 후 수집중 . . .
1일 작성글 검색 후 수집중 . . .
6
31일 작성글 검색 후 수집중 . . .
30일 작성글 검색 후 수집중 . . .
29일 작성글 검색 후 수집중 . . .
28일 작성글 검색 후 수집중 . . .
27일 작성글 검색 후 수집중 . . .
26일 작성글 검색 후 수집중 . . .
25일 작성글 검색 후 수집중 . . .
24일 작성글 검색 후 수집중 . . .
23일 작성글 검색 후 수집중 . . .
22일 작성글 검색 후 수집중 . . .
21일 작성글 검색 후 수집중 . . .
20일 작성글 검색 후 수집중 . . .
19일 작성글 검색 후 수집중 . . .
18일 작성글 검색 후 수집중 .

In [9]:
basic_crawling(18,8,31)

,url,title,date
0,https://blog.naver.com/yoosoojoung/222076684802,이마트 트레이더스 월계점 가성비갑 대형식품,2020.08.31.
1,https://blog.naver.com/yuneyun/222076685632,<백일독서 그물독서#1>마이퍼스트월드: 바퀴,2020.08.31.
2,https://blog.naver.com/jun95990/222076683798,[포스코] 2020 하반기 포스코 채용,2020.08.31.
3,https://blog.naver.com/iloveyou915/222076683007,광주광역시 서구 상무시민공원 시민들의 휴식공간으로 딱이야!,2020.08.31.
4,https://blog.naver.com/mkw728/222076680205,(탄핵 무상)자비의 샘물-55,2020.08.31.
...,...,...,...
13226,https://gout7.tistory.com/1051,하나. 어떻게 생각할 것인가,2020.08.01.
13227,https://blog.naver.com/whynews1/222047767476,[인터뷰] 생각하고 ‘신속’ 행동하는 정치인 한은경 오산시의원,2020.08.01.
13228,https://blog.naver.com/alpepel/222047765522,광주광역시동물병원 고정을 시켜놓고 올림,2020.08.01.
13229,https://blog.naver.com/sheenbee/222047763422,남파랑길 부산 5코스(신평동 교차로~강서구 송정공원),2020.08.01.


In [10]:
crawl_df.drop_duplicates(inplace=True) #중복제거
crawl_df = crawl_df.reset_index(drop=True) #인덱스 초기화 
crawl_df

,url,title,date
0,https://blog.naver.com/yoosoojoung/222076684802,이마트 트레이더스 월계점 가성비갑 대형식품,2020.08.31.
1,https://blog.naver.com/yuneyun/222076685632,<백일독서 그물독서#1>마이퍼스트월드: 바퀴,2020.08.31.
2,https://blog.naver.com/jun95990/222076683798,[포스코] 2020 하반기 포스코 채용,2020.08.31.
3,https://blog.naver.com/iloveyou915/222076683007,광주광역시 서구 상무시민공원 시민들의 휴식공간으로 딱이야!,2020.08.31.
4,https://blog.naver.com/mkw728/222076680205,(탄핵 무상)자비의 샘물-55,2020.08.31.
...,...,...,...
13099,https://gout7.tistory.com/1051,하나. 어떻게 생각할 것인가,2020.08.01.
13100,https://blog.naver.com/whynews1/222047767476,[인터뷰] 생각하고 ‘신속’ 행동하는 정치인 한은경 오산시의원,2020.08.01.
13101,https://blog.naver.com/alpepel/222047765522,광주광역시동물병원 고정을 시켜놓고 올림,2020.08.01.
13102,https://blog.naver.com/sheenbee/222047763422,남파랑길 부산 5코스(신평동 교차로~강서구 송정공원),2020.08.01.


## 네이버만 남기기

In [21]:
link_list = crawl_df['url'].to_list()
print(len(link_list))
find_naver = [i for i in range(len(link_list)) if 'naver' in link_list[i]]
print(len(find_naver))


12530
11904


In [22]:
NB_DF = crawl_df.iloc[find_naver]
NB_DF

,url,title,date
0,https://blog.naver.com/jinidag/222104033228,추석연휴 가볼만한곳 한국민속촌 야외 당일치기 나들이,2020.09.30.
1,https://blog.naver.com/taya2765/222104032204,"동래 가볼만한 곳, 복천박물관",2020.09.30.
2,https://blog.naver.com/yeonjae1213/222104028784,배꼽 빠질 뻔했다,2020.09.30.
3,https://blog.naver.com/sunnybridge_/222104026647,"내가 사랑했던 마을,탬워스(Tamworth)",2020.09.30.
4,https://blog.naver.com/lby56/222104022581,언택트 한가위… ‘꼰대’ 삼촌이 10대 조카에게 보내는 편지,2020.09.30.
...,...,...,...
12525,https://blog.naver.com/chookk7/222076688184,“먹튀자본 MBK 김병주의 땅투기로 안산이 망가진다”,2020.09.01.
12526,https://blog.naver.com/bugman1303/222076686202,머리가 좋은 동물들,2020.09.01.
12527,https://blog.naver.com/sherry8279/222076052955,"<도시는 무엇으로 사는 가, 유현준> 도시의 다채로운 생각과...",2020.09.01.
12528,https://blog.naver.com/butterfly0724/222071335341,[경기도 미술관 카페] 호이안,2020.09.01.


In [23]:
NB_DF.to_csv("./나무문화/2020/나무문화_링크_2020_9월.csv", index=False, encoding='UTF-8')
NB_DF.to_excel("./나무문화/2020/나무문화_링크_2020_9월.xlsx", index=False, encoding='UTF-8')
